<a href="https://colab.research.google.com/github/ucfilho/marquesgabi/blob/master/Calculo_013_out_30_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# carrega as bibliotecas do python

In [0]:
%matplotlib inline
from matplotlib import pyplot as plt
import os 
import cv2
import zipfile
from skimage import data
from skimage import filters
from skimage import exposure
import skimage.feature as sk
import numpy as np
import pandas as pd
from google.colab import files
from sklearn.metrics.cluster import entropy
from sklearn.metrics.cluster import homogeneity_score


In [5]:
!wget http://ftp.feq.ufu.br/ubirajara/Sugar_set_30_2019.zip
#!ls -l trash_nov_22_2018.zip


--2019-10-30 13:31:17--  http://ftp.feq.ufu.br/ubirajara/Sugar_set_30_2019.zip
Resolving ftp.feq.ufu.br (ftp.feq.ufu.br)... 200.19.144.11
Connecting to ftp.feq.ufu.br (ftp.feq.ufu.br)|200.19.144.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19584173 (19M) [application/zip]
Saving to: ‘Sugar_set_30_2019.zip’

Sugar_set_30_2019.z 100%[===================>]  18.68M  5.63MB/s    in 3.3s    

2019-10-30 13:31:21 (5.63 MB/s) - ‘Sugar_set_30_2019.zip’ saved [19584173/19584173]



In [0]:

file_name = zipfile.ZipFile('Sugar_set_30_2019.zip', 'r')
file_name.extractall() 

In [0]:
# all images are resized in variable x, are in gray in variable p and in variable w are dataframe
FILE='Sugar_set_30_2019.zip'
img_name=[]
x=[]
p=[]
w=[]
Size=5 # trocar por 64 qdo acabar a analise

with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)
        #x.append(cv2.imread(name))
        x.append(cv2.resize(cv2.imread(name),(Size,Size)))

nrow=len(img_name)
ncol=Size*Size
pp=np.zeros((nrow,ncol))
for i in range(nrow):
    w.append(cv2.cvtColor(np.array(x[i]), cv2.COLOR_BGR2GRAY))
    pp[i,:]=w[i].ravel()

In [0]:
Nfotos=len(img_name)
GLCM=np.zeros((Nfotos, 14))   # lembrar do ANN_Hydrofoil_Neuronio_set_17_2018.ipynb
Nrow=Size
Ncol=Size
#print(GLCM.shape)
#Soma=np.sum(pp[k,:])
for k in range(Nfotos):
  
  p=pp[k,:].reshape(Size,Size)
  Soma=0
  for i in range(Size):
    for j in range(Size):
      Soma=Soma+p[i,j]
   
  for i in range(Size):
    for j in range(Size):
      p[i,j]=p[i,j]/Soma

  for i in range(Nrow):
    for j in range(Ncol):
      #print("i=%d j=%d k=%d"%(i,j,k))
      GLCM[k,7]=GLCM[k,7]-p[i,j]*np.log(p[i,j]) # entropia
      GLCM[k,0]=GLCM[k,0]+p[i,j]**2 #segundo momento angular
      GLCM[k,1]=GLCM[k,1]+(i-j)**2*p[i,j] #contraste
      GLCM[k,3]=GLCM[k,3]+(i-np.mean(p))**2*p[i,j] #Variancia



In [9]:
pd.set_option('display.max_columns', 30)
print(pd.DataFrame(GLCM))

           0         1    2         3    4    5    6         7    8    9   10  \
0   0.040159  4.006764  0.0  5.873156  0.0  0.0  0.0  3.216867  0.0  0.0  0.0   
1   0.041540  4.282528  0.0  5.762803  0.0  0.0  0.0  3.203164  0.0  0.0  0.0   
2   0.040287  4.030399  0.0  5.979967  0.0  0.0  0.0  3.215219  0.0  0.0  0.0   
3   0.040175  4.015663  0.0  6.027608  0.0  0.0  0.0  3.216685  0.0  0.0  0.0   
4   0.040313  4.043838  0.0  5.656749  0.0  0.0  0.0  3.214975  0.0  0.0  0.0   
5   0.040234  4.056964  0.0  5.948915  0.0  0.0  0.0  3.215969  0.0  0.0  0.0   
6   0.040177  3.895373  0.0  5.944445  0.0  0.0  0.0  3.216637  0.0  0.0  0.0   
7   0.040276  3.931919  0.0  6.036687  0.0  0.0  0.0  3.215371  0.0  0.0  0.0   
8   0.040229  3.993338  0.0  6.066993  0.0  0.0  0.0  3.216015  0.0  0.0  0.0   
9   0.040272  3.979461  0.0  5.888981  0.0  0.0  0.0  3.215601  0.0  0.0  0.0   
10  0.040134  3.934069  0.0  5.729726  0.0  0.0  0.0  3.217205  0.0  0.0  0.0   
11  0.041463  3.928237  0.0 

In [10]:
GLCM=pd.DataFrame(GLCM)
pd.set_option('display.max_columns', 30)
print(GLCM.head(5))

          0         1    2         3    4    5    6         7    8    9   10  \
0  0.040159  4.006764  0.0  5.873156  0.0  0.0  0.0  3.216867  0.0  0.0  0.0   
1  0.041540  4.282528  0.0  5.762803  0.0  0.0  0.0  3.203164  0.0  0.0  0.0   
2  0.040287  4.030399  0.0  5.979967  0.0  0.0  0.0  3.215219  0.0  0.0  0.0   
3  0.040175  4.015663  0.0  6.027608  0.0  0.0  0.0  3.216685  0.0  0.0  0.0   
4  0.040313  4.043838  0.0  5.656749  0.0  0.0  0.0  3.214975  0.0  0.0  0.0   

    11   12   13  
0  0.0  0.0  0.0  
1  0.0  0.0  0.0  
2  0.0  0.0  0.0  
3  0.0  0.0  0.0  
4  0.0  0.0  0.0  


In [11]:
print("ENTROPIA=",entropy(w[0])) # O valor nao bate....
# print("HOMOGENEITY=",homogeneity_score(w[0])) ESTE NAO FUNCIONA....

ENTROPIA= 2.886165178199427


In [12]:
print(entropy(w[0]))

2.886165178199427


In [14]:
'''
#print(Nfotos)

##Then I say 

glcm = sk.greycomatrix(w[0], [1], [0])
#glcm = sk.greycomatrix(x[0], [1],[0])
#print(glcm)
#glcm = feature.greycomatrix(grayImg, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])
print(glcm.shape) # (256, 256, 1, 4)

#rank.entropy(glcm, disk(5)) # throws an error since entropy expects a 2-D array in its arguments

print(glcm)
'''

(256, 256, 1, 1)
[[[[0]]

  [[0]]

  [[0]]

  ...

  [[0]]

  [[0]]

  [[0]]]


 [[[0]]

  [[0]]

  [[0]]

  ...

  [[0]]

  [[0]]

  [[0]]]


 [[[0]]

  [[0]]

  [[0]]

  ...

  [[0]]

  [[0]]

  [[0]]]


 ...


 [[[0]]

  [[0]]

  [[0]]

  ...

  [[0]]

  [[0]]

  [[0]]]


 [[[0]]

  [[0]]

  [[0]]

  ...

  [[0]]

  [[0]]

  [[0]]]


 [[[0]]

  [[0]]

  [[0]]

  ...

  [[0]]

  [[0]]

  [[0]]]]


In [15]:
# glcm = greycomatrix(w[0], [5], [0], 256, symmetric=True, normed=True)

NameError: ignored